<a href="https://colab.research.google.com/github/franciscogonzalez-gal/statistical-learning-2/blob/main/Clase8_SL2_Ej.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nombre**: Francisco González

**Carnet:** 24002914

In [1]:
# ==============================================================================
# Ejercicio de Aprendizaje por Refuerzo: El Aventurero del Tesoro
#
# Este archivo Python simula el "Entorno" y la interaccion del agente (el equipo de alumnos).
# Los alumnos toman decisiones (acciones) y el codigo les devuelve el nuevo estado,
# la recompensa y si el episodio ha terminado.
# ==============================================================================

import numpy as np
import random
import time

# Definicion del Entorno del juego
# 'T': Tesoro (meta)
# 'X': Trampa (penalizacion)
# ' ': Camino vacio
# 'S': Inicio
mapa = np.array([
    [' ', ' ', ' ', 'X', ' '],
    [' ', 'X', ' ', ' ', ' '],
    [' ', ' ', 'X', ' ', ' '],
    ['X', ' ', ' ', ' ', ' '],
    ['S', ' ', 'X', ' ', 'T']
])

class AmbienteAventurero:
    """
    Esta clase simula el "Entorno" del Aprendizaje por Refuerzo, similar a un entorno de IA-Gym.
    """
    def __init__(self, mapa):
        self.mapa = mapa
        self.estado_inicial = (4, 0)  # Fila 4, Columna 0
        self.estado_actual = self.estado_inicial
        self.recompensa = 0
        self.done = False
        self.acciones = {
            'arriba': (-1, 0),
            'abajo': (1, 0),
            'izquierda': (0, -1),
            'derecha': (0, 1)
        }

    def reset(self):
        """
        Resetea el entorno a su estado inicial.
        """
        self.estado_actual = self.estado_inicial
        self.recompensa = 0
        self.done = False
        return self.estado_actual, self.done

    def step(self, accion):
        """
        El corazon del ciclo de RL.
        Recibe una accion, la procesa y devuelve el nuevo estado, la recompensa
        y si el episodio ha terminado.
        """
        movimiento = self.acciones.get(accion)
        if movimiento is None:
            return self.estado_actual, -1, False # Penalizacion por accion invalida

        fila_nueva = self.estado_actual[0] + movimiento[0]
        col_nueva = self.estado_actual[1] + movimiento[1]

        # Verificar limites del mapa
        if not (0 <= fila_nueva < len(self.mapa) and 0 <= col_nueva < len(self.mapa[0])):
            return self.estado_actual, -0.5, False # Penalizacion por chocar con una pared

        self.estado_actual = (fila_nueva, col_nueva)

        casilla = self.mapa[self.estado_actual]

        if casilla == 'T':
            self.recompensa = 10  # Gran recompensa por el tesoro
            self.done = True
        elif casilla == 'X':
            self.recompensa = -5  # Gran penalizacion por la trampa
            self.done = True
        else:
            self.recompensa = -0.1 # Pequena penalizacion por cada paso (costo de tiempo)

        return self.estado_actual, self.recompensa, self.done

    def render(self):
        """
        Dibuja el mapa para que el equipo pueda ver su progreso.
        """
        mapa_visual = self.mapa.copy().astype('<U1')
        fila, col = self.estado_actual
        mapa_visual[fila, col] = 'A' # 'A' para el Aventurero

        print("\nEstado Actual del Laberinto:")
        for row in mapa_visual:
            print(" ".join(row))
        print("--------------------")

# ==============================================================================
# Bucle de Interaccion del Ejercicio
# ==============================================================================
def iniciar_juego():
    env = AmbienteAventurero(mapa)
    estado, done = env.reset()

    print("¡Bienvenido a 'El Aventurero del Tesoro'!")
    print("Tu mision es guiar al agente 'A' para encontrar el tesoro 'T' evitando las trampas 'X'.")
    print("Acciones disponibles: 'arriba', 'abajo', 'izquierda', 'derecha'.")
    print("Escribe 'fin' para terminar el juego.")
    env.render()

    recompensa_total = 0
    pasos = 0

    while not done:
        accion = input("Elige tu proxima accion (arriba, abajo, izquierda, derecha): ").lower()
        if accion == 'fin':
            print("Juego terminado por el usuario.")
            break

        estado, recompensa, done = env.step(accion)
        env.render()
        recompensa_total += recompensa
        pasos += 1

        print(f"Recompensa recibida: {recompensa}")
        print(f"Recompensa total acumulada: {recompensa_total:.1f}")

    if 'T' in env.mapa[estado]:
        print(f"\n¡Felicidades, encontraste el tesoro en {pasos} pasos!")
    elif 'X' in env.mapa[estado]:
        print(f"\nCaiste en una trampa... Juego terminado.")

    print(f"Recompensa final: {recompensa_total:.1f}")
    print("--------------------")
    print("Análisis post-juego: ¿Qué estrategia siguió tu equipo?")

if __name__ == "__main__":
    iniciar_juego()


¡Bienvenido a 'El Aventurero del Tesoro'!
Tu mision es guiar al agente 'A' para encontrar el tesoro 'T' evitando las trampas 'X'.
Acciones disponibles: 'arriba', 'abajo', 'izquierda', 'derecha'.
Escribe 'fin' para terminar el juego.

Estado Actual del Laberinto:
      X  
  X      
    X    
X        
A   X   T
--------------------
Elige tu proxima accion (arriba, abajo, izquierda, derecha): arriba

Estado Actual del Laberinto:
      X  
  X      
    X    
A        
S   X   T
--------------------
Recompensa recibida: -5
Recompensa total acumulada: -5.0

Caiste en una trampa... Juego terminado.
Recompensa final: -5.0
--------------------
Análisis post-juego: ¿Qué estrategia siguió tu equipo?


# Análisis post-juego: ¿Qué estrategia siguió tu equipo?

El equipo no siguió una política clara, ya que la primera acción fue "arriba" y llevó directamente a una trampa.  
La estrategia fue **miopía**: se pensó en el movimiento inmediato sin considerar el valor futuro ni las posibles consecuencias.  

Una mejor estrategia habría sido:  
- Evitar movimientos hacia celdas de riesgo inmediato.  
- Planificar una ruta hacia el tesoro aceptando pequeñas penalizaciones por pasos.  
- Valorar el largo plazo sobre la ganancia inmediata, aplicando la lógica de la **Ecuación de Bellman**.  


1. Agente y Entorno:

- ¿Quién representaba al Agente en este ejercicio?

- ¿Qué elementos del código y el juego representaban el Entorno?

2. Estado, Acción y Recompensa:

- Describa un Estado en el juego. ¿Qué información te daba?

- ¿Cuáles eran las Acciones? ¿Y la Política de tu equipo?

- ¿Cómo se usaron las Recompensas para guiar sus decisiones? ¿Qué diferencia hubo entre una recompensa por paso y una recompensa por objetivo?

3. La Ecuación de Bellman:

- Cuando estaban decidiendo qué hacer, ¿pensaron solo en la recompensa del siguiente paso?

- Si encontraron el tesoro, ¿cree que la mejor decisión inmediata fue la que los llevó a él? ¿Podrían haber tomado una penalización pequeña para obtener una gran recompensa después? Esto es el concepto de pensar en el valor a largo plazo, que es el corazón de la Ecuación de Bellman.

# 1. Agente y Entorno
- **Agente:** el jugador/equipo que decide las acciones.  
- **Entorno:** el mapa, las reglas de movimiento, las recompensas y la clase `AmbienteAventurero`.  


# 2. Estado, Acción y Recompensa
- **Estado:** posición actual del agente en la grilla.  
- **Acciones:** arriba, abajo, izquierda, derecha.  
- **Política:** estrategia usada para elegir acciones.  
- **Recompensas:** penalizan pasos y trampas, premian el tesoro.  



# 3. Ecuación de Bellman
- **Recompensa inmediata:** puede llevar a errores (ejemplo: caer en trampa).  
- **Valor a largo plazo:** aceptar pequeñas penalizaciones para alcanzar la gran recompensa final (tesoro).  
